# Using Ray to Execute Parellel Code

For more details about Ray refer to <https://docs.ray.io/en/latest/ray-core/walkthrough.html>.

In [ ]:
import time

import ray

Initialize a local Ray cluster.

In [ ]:
ray.init()

Define two functions that do the same thing. The first function does not use Ray, each iteration will run one after the other. The second function uses Ray. The iterations can run in parellel.

In [ ]:
def normal_function(x):
    print(f"Iteration {x}")
    time.sleep(1)
    return x


@ray.remote
def ray_function(x):
    print(f"Iteration {x}")
    time.sleep(1)
    return x


number_of_iterations = 5

Test how long it takes to run the function in a **single thread/process**.

In [ ]:
%%time
values = [normal_function(i) for i in range(number_of_iterations)]
print(values)

Test how long it takes to run the function using **Ray**.

In [ ]:
%%time
futures = [ray_function.remote(i) for i in range(number_of_iterations)]
values = ray.get(futures)
print(values)